In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style

print(plt.style.available)
%matplotlib inline

driver=webdriver.Edge()

url='https://tj.zu.fang.com/house-a037-b0154/'
driver.get(url)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [2]:
string_list=[]
i=0
while i<2:
    table=driver.find_element(By.CLASS_NAME,'houseList')
    # Extract table rows
    rows = table.find_elements(By.TAG_NAME, 'dl')

    # Initialize lists to store data
    data = []

    # Loop through rows and extract data
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'dd')
        if cells:  # Skip rows without data (e.g., headers)
            row_data = [cell.text for cell in cells]
            data.append(row_data)

    df = pd.DataFrame(data)
    string_list.append(df)
    #driver.refresh()
    try:
        next_page=driver.find_element(By.LINK_TEXT,'下一页')
        next_page.click()
        i+=1
    except NoSuchElementException:
        break

In [3]:
def split_info(string_list,index_num):
    info = string_list[0][0][index_num].split("\n")
    #数据清洗
    title = info[0]
    price = info[-1]
    frequency = info[1].split("|")[0]
    type =  info[1].split("|")[1]
    square =  info[1].split("|")[2]
    direction =  info[1].split("|")[3]
    district =  info[2]
    traffic =  info[3]
    return [title,price,frequency,type,square,direction,district,traffic]


In [4]:
df = pd.DataFrame(columns=["标题","价格","租赁形式","户型","面积","朝向","区域","交通"])
for i in range(len(string_list)):#遍历子列表 写入数据
    try:
        for j in range(len(string_list[i])):
            if j>59:
                i+=1
            infomation = split_info(string_list,j)
            new_row = pd.DataFrame([infomation], columns=df.columns)
            df = pd.concat([df, new_row], ignore_index=True)
    except Exception as e:
        print(f"处理第 {i+1} 个子列表的第{j+1}条数据时出错: {e}")

In [5]:
df

,标题,价格,租赁形式,户型,面积,朝向,区域,交通
0,整租·招商津湾天玺 2室1厅 西南,10620元/月,整租,2室1厅,174㎡,朝西南,和平-劝业场街-招商津湾天玺,距3号线津湾广场站约393米。
1,整租·中海和平之门 2室1厅 南,6500元/月,整租,2室1厅,66㎡,朝南,和平-劝业场街-中海和平之门,距4号线金街站约457米。
2,整租·津门公寓 3室2厅 东南/西北,23000元/月,整租,3室2厅,279㎡,朝西北,和平-劝业场街-津门公寓,距4号线金街站约729米。
3,整租·招商津湾天玺 2室2厅 南,12000元/月,整租,2室2厅,174㎡,朝南,和平-劝业场街-招商津湾天玺,距3号线津湾广场站约393米。
4,整租·招商津湾天玺 2室1厅 南,12000元/月,整租,2室1厅,119㎡,朝南,和平-劝业场街-招商津湾天玺,距3号线津湾广场站约393米。
...,...,...,...,...,...,...,...,...
87,整租·宜天花园 2室1厅 西南,4100元/月,整租,2室1厅,67㎡,朝西南,和平-劝业场街-宜天花园,距1号线海光寺站约612米。
88,整租·南京路(和平区) 3室1厅 南,4500元/月,整租,3室1厅,86㎡,朝南,和平-劝业场街-南京路250号楼,生活便利低价出租紧急出租
89,整租·和平翰林公馆 1室1厅 北,4200元/月,整租,1室1厅,60㎡,朝北,和平-劝业场街-和平翰林公馆,距1号线营口道站约78米。
90,整租·和平翰林公馆 1室1厅 东,4000元/月,整租,1室1厅,60㎡,朝东,和平-劝业场街-和平翰林公馆,距1号线营口道站约78米。


In [7]:
df.to_csv("queyechang_rent_data.csv")